In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [60]:
df = pd.read_csv('train.csv')

In [61]:
df.head()

,Id,0,1,2,3,4,5,6,7,8,Category
0,0,50,21,77,0,28,0,27,48,22,2
1,1,55,0,92,0,0,26,36,92,56,4
2,2,53,0,82,0,52,-5,29,30,2,1
3,3,37,0,76,0,28,18,40,48,8,1
4,4,37,0,79,0,34,-26,43,46,2,1


In [62]:
df.shape

(43500, 11)

In [63]:
# Define X and y
feature_cols=['0','1','2', '3', '4', '5', '6', '7', '8']
X = df[feature_cols]
y= df.Category

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Upsampling on X_train

In [64]:
iclass1 = np.where(y_train==1)[0]
iclass2 = np.where(y_train==2)[0]
iclass3 = np.where(y_train==3)[0]
iclass4 = np.where(y_train==4)[0]

In [65]:
n_iclass1=len(iclass1)
n_iclass2 = len(iclass2)
n_iclass3 = len(iclass3)
n_iclass4 = len(iclass4)

In [66]:
print({'1':n_iclass1, '2': n_iclass2, '3': n_iclass3, '4:': n_iclass4})

{'1': 25653, '2': 28, '3': 97, '4:': 5023}


In [67]:
i_class2_upsampled = np.random.choice(iclass2, size=n_iclass1, replace=True)
i_class3_upsampled = np.random.choice(iclass3, size = n_iclass1, replace = True)
i_class4_upsampled = np.random.choice(iclass4, size = n_iclass1, replace = True)

In [68]:
np.concatenate((y[i_class2_upsampled], y[iclass1]))
np.concatenate((y[i_class3_upsampled], y[iclass1]))
np.concatenate((y[i_class4_upsampled], y[iclass1]))

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [69]:
print({'1:': len(iclass1), '2:': len(i_class2_upsampled), '3': len(i_class3_upsampled), '4:': len(i_class4_upsampled)})

{'1:': 25653, '2:': 25653, '3': 25653, '4:': 25653}


In [70]:
p1 = np.append(iclass1, i_class2_upsampled)
pt2 = np.append(p1, i_class3_upsampled)
pf = np.append(pt2, i_class4_upsampled)
pf = pd.DataFrame(pf)

In [71]:
pf.head()

,0
0,0
1,1
2,2
3,3
4,4


In [72]:
df_new = df.join(pf)

In [73]:
df_new.head()

,Id,0,1,2,3,4,5,6,7,8,Category,0
0,0,50,21,77,0,28,0,27,48,22,2,0
1,1,55,0,92,0,0,26,36,92,56,4,1
2,2,53,0,82,0,52,-5,29,30,2,1,2
3,3,37,0,76,0,28,18,40,48,8,1,3
4,4,37,0,79,0,34,-26,43,46,2,1,4


## Build a Support Vector Machine Model

In [74]:
from sklearn.ensemble import RandomForestClassifier

In [75]:
# Instantiate
rf = RandomForestClassifier()

# Fit the model
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

C:\Users\allis\Anaconda3\envs\py36\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Evaluate Model

In [76]:
score_up = metrics.accuracy_score(y_pred, y_test)
print(score_up)

0.9997241379310344


## Make Submission

In [77]:
test = pd.read_csv('test.csv')

In [78]:
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [79]:
X_sub = test[feature_cols]

Category = rf.predict(X_sub)

In [80]:
sub = pd.DataFrame(Category)
sub.head()

,0
0,4
1,4
2,1
3,4
4,1


In [81]:
sub.rename(columns={0:'Category'}, inplace=True)
sub.index.names = ['Id']

In [82]:
sub.head()

,Category
Id,
0,4
1,4
2,1
3,4
4,1


In [83]:
sub.to_csv('Submission.csv')